In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sportsball
from sportsball.nfl.nfl import Season, Team, Game
from sportsball.parsing import MasseyParser
from gspread_pandas import Spread, Client
import pymongo

In [ ]:
client = pymongo.MongoClient()

In [3]:
season_2018 = Season(
    year=2018,
    sheettitle='NFL 2018 Expected Wins',
)

In [18]:
db = client.sportsball

In [19]:
teams = db.teams

In [30]:
db.teams.create_index([('id', pymongo.TEXT)], unique=True)

'id_text'

## Inserting seasons

In [ ]:
result = db.seasons.insert_one({'year': 2018})

## Inserting teams

In [46]:
result = db.teams.insert_many([{
    'id': team.team_id,
    'conference': team.conference,
    'division': team.division,
    'year': team.season.year,
} for team in season_2018.teams.values()])

## Inserting games

In [82]:
for game in season_2018.games:
    away_team_id = db.teams.find_one({'id': game.away_team.team_id})
    home_team_id = db.teams.find_one({'id': game.home_team.team_id})
    obj = {
        'away_score': game.away_score,
        'home_score': game.home_score,
        'away_team_id': away_team_id['_id'],
        'home_team_id': home_team_id['_id'],
        'game_round': game.game_round.value,
        'neutral': game.neutral,
        'played': game.played,
        'playoff': game.playoff,
        'week': game.week,
        'season_id': ssn['_id'],
    }
    db.games.insert_one(obj)

In [3]:
from sportsball.parsing import SagarinParser

In [88]:
parser = SagarinParser('../chromedriver')

In [90]:
tree = parser.parse_nfl(6)

In [94]:
select1 = tree.cssselect('.sagarin-page > br+ pre')

In [101]:
from bs4 import BeautifulSoup

In [107]:
pattern = r'\s\d{1,2}\s+([\da-zA-Z\s]+)\s+='

In [108]:
import re

In [109]:
result = re.search(pattern=pattern, string='\xa0  1  Los Angeles Rams        =')

In [117]:
teamnames = []
for x in tree.cssselect('#sagarin .sagarin-page br+ font font br+ font'):
    if re.search(pattern=r'^_+$', string=x.text):
        break
    result = re.search(pattern=pattern, string=x.text)
    if result:
        teamnames.append(result.group(1).strip())

In [122]:
rating_pattern=r'(\d{1,2}\.\d{1,2})$'
ratings = []
for x in tree.cssselect('font:nth-child(66) , font:nth-child(59) , font:nth-child(52) , font:nth-child(45) , font:nth-child(38) , font:nth-child(31) , font:nth-child(24) , font:nth-child(17) , font:nth-child(10) , br~ font+ font:nth-child(3)'):
    result = re.search(pattern=rating_pattern, string=x.text)
    if result:
        ratings.append(float(result.group(1).strip()))
        

In [124]:
ratings = ratings[:len(teamnames)]

In [126]:
import pandas

In [127]:
datatable = pandas.DataFrame({'team': teamnames, 'rating': ratings})

In [4]:
parser = SagarinParser('../chromedriver')

In [9]:
datatable = parser.parse_nfl(week=6)

In [5]:
parser.update_spreadsheet(week=6)

In [7]:
parser = MasseyParser('../chromedriver')

In [8]:
parser.update_spreadsheet(week=6)